In [1]:
import sqlite3
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Fetch Data from SQLite Database

In [2]:
#This function loads ad campaign data from an SQLite database.
#If an error occurs (e.g., table not found), it prints an error message.
#It ensures the connection is closed even if an error happens

def fetch_data():
    """Retrieve data from the SQLite database."""
    conn = sqlite3.connect("ad_campaigns.db")
    try:
        df = pd.read_sql("SELECT * FROM ad_campaigns", conn)
        print("Data loaded successfully.")
        return df
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None
    finally:
        conn.close()

# Preprocess Data

In [3]:
#Preprocess Data
def preprocess_data(df):
    """Cleans and preprocesses ad campaign data."""
    #Step 1: Handle Empty DataFrames
    if df is None or df.empty:
        print(" Error: DataFrame is empty!")
        return None  
        
# Step 2: Remove Missing Values
# Drops any rows with missing values (NaN).
# Creates a copy of the DataFrame to prevent modifying the original.

    df = df.dropna().copy()  # Ensure we work on a copy

# Step 3: Encode Categorical Variables
# Identifies categorical columns (Platform, Content_Type, etc.).
# Uses LabelEncoder() to convert categorical values into numbers.
# Stores the encoders in a dictionary (encoders[col] = le) for later use

    # Encode categorical variables
    categorical_cols = ['Platform', 'Content_Type', 'Target_Age', 'Target_Gender', 'Region']
    existing_categorical_cols = [col 
                                 for col in categorical_cols 
                                 if col in df.columns]

    encoders = {}  # Store encoders for reuse in predictions
    for col in existing_categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        encoders[col] = le  # Save encoder
        
# Step 4: Scale Numerical Features
#Identifies numerical columns (Budget, Duration, etc.).
# Uses MinMaxScaler() to normalize the data between 0 and 1.
# If no numerical columns exist, sets scaler = None.

    #  Scale numerical features
    numerical_cols = ['Budget', 'Duration', 'Clicks', 'Conversions', 'CTR', 'CPC', 'Conversion_Rate']
    existing_numerical_cols = [col 
                               for col in numerical_cols 
                               if col in df.columns]

    if existing_numerical_cols:
        scaler = MinMaxScaler()
        df[existing_numerical_cols] = scaler.fit_transform(df[existing_numerical_cols])
    else:
        scaler = None  # No scaling if no numerical columns exist
        
# Save encoders & scaler for reuse in predictions
#Saves categorical encoders (label_encoders.pkl) for reuse during predictions.
# Saves the numerical scaler (scaler.pkl) so the test data can be transformed in the same way
    
    with open("label_encoders.pkl", "wb") as f:
        pickle.dump(encoders, f)

    if scaler:
        with open("scaler.pkl", "wb") as f:
            pickle.dump(scaler, f)

    print(" Preprocessing completed successfully.")
    return df

# Execute the Script

In [4]:
#Step 5: Execute the Script
# Loads the dataset from SQLite.
# Calls preprocess_data(df) to clean and transform the data.
# Saves the cleaned dataset (cleaned_data.csv).
# Catches and prints any errors that occur


if __name__ == "__main__":
    try:
        df = fetch_data()
        df_cleaned = preprocess_data(df)

        if df_cleaned is not None:
            df_cleaned.to_csv("cleaned_data.csv", index=False)
            print(" Preprocessed data saved as 'cleaned_data.csv'")
        else:
            raise ValueError(" Preprocessing failed. No data to save.")
    except Exception as e:
        print(f" Error during preprocessing: {e}")

Data loaded successfully.
 Preprocessing completed successfully.
 Preprocessed data saved as 'cleaned_data.csv'
